# Project 2: Red vs. Blue Encryption Method

**SUMMARY:** In this project we will be setting up the device to take in different plain text inputs, the device will encrypt these inputs and we will read back the encrypted text.

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWNANO'
CRYPTO_TARGET = 'NONE'
SS_VER = 'SS_VER_2_1'

%run "Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [2]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd hardware/victims/firmware/simpleserial-base
make clean
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1


.././hal/Makefile.hal:250: *** Invalid or empty PLATFORM: NONE. Known platforms: 
+-------------------------------------------------------+
| PLATFORM      | DESCRIPTION                           |
+=======================================================+
| AVR/XMEGA Targets (8-Bit RISC)                        |
+=======================================================+
+-------------------------------------------------------+
| CWLITEXMEGA   | CW-Lite XMEGA (Alias for CW303)       |
+-------------------------------------------------------+
| CW301_AVR     | Multi-Target Board, AVR Target        |
+-------------------------------------------------------+
| CW303         | XMEGA Target (CWLite), Also works     |
|               | for CW308T-XMEGA                      |
+-------------------------------------------------------+
| CW304         | ATMega328P (NOTDUINO), Also works     |
|               | for CW308T-AVR                        |
+-----------------------------------------------

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
arm-none-eabi-gcc (GNU Arm Embedded Toolchain 10-2020-q4-major) 10.2.1 20201103 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Welcome to another exciting ChipWhisperer target build!!
.
Assembling: .././hal/stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CWNANO/stm32f0_startup.lst -I.././simpleserial/ -I.././simpleserial/ -I.././hal -I.././hal/stm32f0 -I.././hal/stm32f0/CMSIS -I.././hal/stm32f0/CMSIS/core -I.././hal/stm32f0/CMSIS/device -I.././hal/stm32f0/Legacy .././hal/stm32f0/stm32f0_startup.S -o objdir-CWNANO/stm3

In [3]:
cw.program_target(scope, prog, "hardware/victims/firmware/simpleserial-base/simpleserial-base-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 14439 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 14439 bytes


In [4]:
from tqdm import tnrange
import aes
import numpy as np
import time

#Enter the system key once per program. If you want to set another key, you will have to re-program the board above

#transposes a matrix 
def transpose_matrix(lst):
    matrix = [lst[i:i + 4] for i in range(0, len(lst), 4)]
    transposed = [[matrix[j][i] for j in range(4)] for i in range(4)]
    return [elem for row in transposed for elem in row]

key_input = input("Enter a 16 character Key")
ky = bytearray(key_input.encode('utf-8'))

while len(ky) % 16 != 0:
    ky.append(0)
#transpose matrix for internal AES implmentation
ky_transpose = transpose_matrix(ky)
print("Set the Key to: ", [hex(num) for num in ky_transpose])
cipher = aes.aes(int.from_bytes(ky_transpose, byteorder='big'), 128)
#Send key to Nano
target.simpleserial_write('p', ky)
target.simpleserial_read('r', 16)

Enter a 16 character Key1234567891234567
Set the Key to:  ['0x31', '0x35', '0x39', '0x34', '0x32', '0x36', '0x31', '0x35', '0x33', '0x37', '0x32', '0x36', '0x34', '0x38', '0x33', '0x37']


CWbytearray(b'31 32 33 34 35 36 37 38 39 31 32 33 34 35 36 37')

In [5]:
# ###After programming target, we will write several plain texts to it###
# ##First write the plain text, then get the encrypted text back###
# from tqdm import tnrange
# import aes
# import numpy as np
# import time


# pt = bytearray([])
# ct = bytearray([])

# #for user input

# data = input("Enter a 16 character String")
# pt =bytearray(data.encode('utf-8'))

# #debug plaintext and ciphertext seen in FIPS test cases
# #pt = bytearray([0x32,0x88, 0x31, 0xe0, 0x43, 0x5a, 0x31, 0x37, 0xf6, 0x30, 0x98, 0x07, 0xa8, 0x8d, 0xa2, 0x34])
# #ky = bytearray([0x2b, 0x28, 0xab, 0x09, 0x7e, 0xae, 0xf7, 0xcf, 0x15, 0xd2, 0x15, 0x4f, 0x16, 0xa6, 0x88, 0x3c])

# #If the data is not 16 characters, pad it with zeros
# while len(pt) % 16 != 0:
#     pt.append(0)
# print("You entered: ", data)
# print("Input in Bytes: ",pt )
# #iterate 16 bytes at a time through the entire message 
# for i in range(0, len(pt), 16):
#     pt_chunk = pt[i:i+16]
#     print("Sending 16 bytes of Plaintext: ", pt_chunk)
#     target.simpleserial_write('p', pt_chunk)
#     #Read the ciphertext
#     response = target.simpleserial_read('r', 16)
#     ct[i:i+16] = transpose_matrix(response)
#     print("Recieved 16 bytes of CipherText: ", response)
#     #Decode the ciphertext
#     plaintext = cipher.dec_once(int.from_bytes(ct[i:i+16], byteorder='big'))
#     decrypted_bytes = [hex(num) for num in plaintext]
#     #Print the raw bytes and the ASCII Bytes
#     print("Decrytped Data: ", decrypted_bytes)
#     decrypted_bytes_transpose = transpose_matrix(decrypted_bytes)
#     print("Decrypted ASCII ", ''.join(chr(int(num, 16)) for num in decrypted_bytes_transpose))



    

In [6]:
sbox = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f 
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

def aes_internal(inputdata, key):
    return sbox[inputdata ^ key]

In [7]:
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

key, text = ktp.next()

target.set_key(key)

N = 2500
for i in tnrange(N, desc='Capturing traces'):
    scope.arm()
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    key, text = ktp.next() 
assert(len(trace_array) == 2500)
print("✔️ OK to continue!")

C:\Users\alexb\AppData\Local\Temp\ipykernel_115380\342022116.py:14: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(N, desc='Capturing traces'):


Capturing traces:   0%|          | 0/2500 [00:00<?, ?it/s]

✔️ OK to continue!


In [17]:
from tqdm import tnrange
import numpy as np

def calculate_diffs(guess, byteindex=0, bitnum=0):
    """Perform a simple DPA on two traces, uses global `textin_array` and `trace_array` """
    
    one_list = []
    zero_list = []

    for trace_index in range(numtraces):
        hypothetical_leakage = aes_internal(guess, textin_array[trace_index][byteindex])

        #Mask off the requested bit
        if hypothetical_leakage & (1<<bitnum):
            one_list.append(trace_array[trace_index])
        else:
            zero_list.append(trace_array[trace_index])

    one_avg = np.asarray(one_list).mean(axis=0)
    zero_avg = np.asarray(zero_list).mean(axis=0)
    return abs(one_avg - zero_avg)

numtraces = np.shape(trace_array)[0] #total number of traces
numpoints = np.shape(trace_array)[1] #samples per trace

#Store your key_guess here, compare to known_key
key_guess = []
known_key = [0x31, 0x35, 0x39, 0x34, 0x32, 0x36, 0x31, 0x35, 0x33, 0x37, 0x32, 0x36, 0x34, 0x38, 0x33, 0x37]

for subkey in tnrange(0, 16, desc="Attacking Subkey"):

    mean_diffs = []

    for key_byte_guess_value in range(255):

        one_list = []
        zero_list = []

        hypothetical_leakage = []

        for trace_index in range(numtraces):
            input_byte = textin_array[trace_index][subkey]

            #Get a hypothetical leakage list - use aes_internal(guess, input_byte)   

            hypothetical_leakage.append(aes_internal(input_byte, key_byte_guess_value) & 0x01)

            if hypothetical_leakage[trace_index] == 1:
                one_list.append(trace_array[trace_index])
            else:
                zero_list.append(trace_array[trace_index])

        one_avg = np.mean(one_list,axis=0)
        zero_avg = np.mean(zero_list,axis=0)

        max_diff_value = np.max(np.abs(one_avg - zero_avg))

        if(max_diff_value >= 0.006):
            key_guess.append(hex(key_byte_guess_value))
        
print(key_guess)

C:\Users\alexb\AppData\Local\Temp\ipykernel_115380\477790084.py:30: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for subkey in tnrange(0, 16, desc="Attacking Subkey"):


Attacking Subkey:   0%|          | 0/16 [00:00<?, ?it/s]

['0xd0', '0xa7', '0xb2', '0xbf', '0x81', '0x73', '0xc0', '0x97', '0x27', '0xc3', '0xfd', '0xaf', '0x49', '0xe8', '0xe3', '0xcf', '0xfb']
